# TERM PAPER TEAM DVD TECH-2

## Exercise 1

In [ ]:
#importing all the libraries needed for the tasks.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

### Part 1- reading in the data

In [11]:
DATA_PATH = 'data'
data = []
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
years = ['2014','2015','2016','2017','2018','2019','2020','2021','2022','2023','2024']
for x in years:
    for y in months:
        filename = f'{DATA_PATH}/SCE-{y}-{x}.csv'
        print(f'processing file {filename}')
        df = pd.read_csv(filename,parse_dates=True)



processing file data/SCE-Jan-2014.csv
processing file data/SCE-Feb-2014.csv
processing file data/SCE-Mar-2014.csv
processing file data/SCE-Apr-2014.csv
processing file data/SCE-May-2014.csv
processing file data/SCE-Jun-2014.csv
processing file data/SCE-Jul-2014.csv
processing file data/SCE-Aug-2014.csv
processing file data/SCE-Sep-2014.csv
processing file data/SCE-Oct-2014.csv
processing file data/SCE-Nov-2014.csv
processing file data/SCE-Dec-2014.csv
processing file data/SCE-Jan-2015.csv
processing file data/SCE-Feb-2015.csv
processing file data/SCE-Mar-2015.csv
processing file data/SCE-Apr-2015.csv
processing file data/SCE-May-2015.csv
processing file data/SCE-Jun-2015.csv
processing file data/SCE-Jul-2015.csv
processing file data/SCE-Aug-2015.csv
processing file data/SCE-Sep-2015.csv
processing file data/SCE-Oct-2015.csv
processing file data/SCE-Nov-2015.csv
processing file data/SCE-Dec-2015.csv
processing file data/SCE-Jan-2016.csv
processing file data/SCE-Feb-2016.csv
processing f